In [61]:
import pandas as pd

In [62]:
df_final = pd.read_csv('data/datasetFinal/cobros_final.csv')
df_pagare = pd.read_csv('data/datasetFinal/Capital_Pagare_FechaAperturaCredito.csv').rename(columns= {"idcredito":"idCredito"})

/var/folders/tl/n2n7gv595bq_stcnjf2_yz440000gn/T/ipykernel_62120/3030482662.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_final = pd.read_csv('data/datasetFinal/cobros_final.csv')


In [63]:
df_2025 = df_final[df_final["fechaEnvioCobro"]> "2025-01-01"]
df_final = df_final[df_final["fechaEnvioCobro"]<= "2025-01-01"]

## 2025

In [64]:
df_2025_grouped = df_2025.groupby("idCredito").agg({"montoCobrado":"sum"}).rename(columns={"montoCobrado":"montoCobrado2025"}).reset_index()

## 2022 - 2024

In [65]:
df_final_grouped = df_final.groupby('idCredito').agg(
    montoExigible=('montoExigible', 'sum'),
    montoCobrar=('montoCobrar', 'sum'),
    montoCobrado=('montoCobrado', 'sum'),
    vecesPagadas=('montoCobrado', 'sum'),
    totalCobros=('montoCobrado', 'count'),
    ultimoCobro=('fechaEnvioCobro', 'max')
).reset_index()
df_final_grouped['proporcionPagos'] = df_final_grouped['vecesPagadas'] / df_final_grouped['totalCobros']

In [66]:
df_final_grouped.head()

,idCredito,montoExigible,montoCobrar,montoCobrado,vecesPagadas,totalCobros,ultimoCobro,proporcionPagos
0,6,375.47,375.47,0.00,0.00,1,14/08/2023,0.000000
1,3674,344.18,344.18,0.00,0.00,1,10/05/2024,0.000000
2,4635,1441.17,1441.17,160.13,160.13,9,13/03/2023,17.792222
3,4914,3949.80,3949.80,384.98,384.98,10,13/03/2023,38.498000
4,8947,2316.06,2316.06,0.00,0.00,9,13/03/2023,0.000000


In [70]:
df_final_joined = df_final_grouped.merge(df_pagare, on='idCredito', how='inner')

In [71]:
df_final_joined_2025 = df_final_joined.merge(df_2025_grouped, on='idCredito', how='left')
df_final_joined_2025['proporcionPagada2025'] = df_final_joined_2025['montoCobrado2025'] / df_final_joined_2025['pagare']

In [75]:
df_final_joined_2025.to_csv('data/datasetFinal/df_final_joined_2025.csv', index=False)